In [7]:
import tkinter as tk
from tkinter import messagebox
import cv2
from threading import Thread
import time

class VideoCaptureApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.video_source = 0
        self.vid = cv2.VideoCapture(self.video_source)
        self.canvas = tk.Canvas(window, width=self.vid.get(cv2.CAP_PROP_FRAME_WIDTH), height=self.vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
        self.canvas.pack()

        self.btn_record = tk.Button(window, text="Record", width=50, command=self.record)
        self.btn_record.pack(anchor=tk.CENTER, expand=True)

        self.btn_reset = tk.Button(window, text="Reset", width=50, command=self.reset)
        self.btn_reset.pack(anchor=tk.CENTER, expand=True)

        self.recording = False
        self.out = None
        self.start_time = None
        self.video_file = "output.avi"

        self.delay = 15
        self.update()
        self.window.mainloop()

    def record(self):
        if not self.recording:
            self.recording = True
            self.start_time = time.time()
            fourcc = cv2.VideoWriter_fourcc(*'XVID')
            self.out = cv2.VideoWriter(self.video_file, fourcc, 20.0, (int(self.vid.get(3)), int(self.vid.get(4))))
            self.btn_record.config(text="Recording...")
            self.record_video()

    def record_video(self):
        if self.recording:
            ret, frame = self.vid.read()
            if ret:
                if time.time() - self.start_time <= 30:
                    self.out.write(frame)
                    self.photo = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    self.photo = cv2.resize(self.photo, (self.canvas.winfo_width(), self.canvas.winfo_height()))
                    self.photo = tk.PhotoImage(master=self.canvas, data=cv2.imencode('.ppm', self.photo)[1].tobytes())
                    self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
                    self.window.after(self.delay, self.record_video)
                else:
                    self.stop_recording()

    def reset(self):
        if self.recording:
            self.stop_recording()
        if self.out:
            self.out.release()
            self.out = None
        self.canvas.delete("all")
        self.btn_record.config(text="Record")
        self.video_file = "output.avi"

    def stop_recording(self):
        if self.recording:
            self.recording = False
            if self.out:
                self.out.release()
            self.out = None
            self.btn_record.config(text="Record")
            messagebox.showinfo("Info", "Video recording stopped after 30 seconds.")

    def update(self):
        if not self.recording:
            ret, frame = self.vid.read()
            if ret:
                self.photo = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                self.photo = cv2.resize(self.photo, (self.canvas.winfo_width(), self.canvas.winfo_height()))
                self.photo = tk.PhotoImage(master=self.canvas, data=cv2.imencode('.ppm', self.photo)[1].tobytes())
                self.canvas.create_image(0, 0, image=self.photo, anchor=tk.NW)
        self.window.after(self.delay, self.update)

    def __del__(self):
        if self.vid.isOpened():
            self.vid.release()
        if self.out:
            self.out.release()

# Create a window and pass it to the Application object
root = tk.Tk()
app = VideoCaptureApp(root, "Tkinter OpenCV Video Capture")
